# Prerequisites

- ECDLP
- Elliptic curves
- Pairings

# Theory

**Task**: Reduce the ECDLP to an easier problem (Ex: an easier group)

*Idea*: Use the Weil/Tate pairing to convert a discrete log problem in $E(\mathbb{F}_p)$ to one in $\mathbb{F}^\times_{p^k}$

Since discrete log problems in finite fields can be attacked by index calculus (or other) methods, they can be solved faster than elliptic curve discrete log problems, as long as the field $\mathbb{F}_{p^k}$ is not much larger than $\mathbb{F}_p$

Recall that for an elliptic curve $E$ defined over $\mathbb{F}_p$ 
- we let $E[m]$ denote the set of points of order dividing $m$ with coordinates in the algebraic closure. 
- If $\gcd(p, m)=1$ and $S, T\in E[m]$, then the Weil pairing $e_m(S, T)$ is an $m$th root of unity and can be computed fairly quickly.  
- The pairing is bilinear, and if $\{S, T\}$ is a basis for $E[m]$, then $e_m(S, T)$ is a primitive $m$th root of unity.
- For any $S => e_m(S, S) = 1$.

There exists $n$ such that $Q=nP \iff mQ=\mathcal{O}$ and the Weil paring $e_m(P, Q)=1$.

- Let $E$ be an elliptic curve over $\mathbb{F}_p$. 
- Let $P, Q\in E(\mathbb{F}_p)$. 
- Let $m$ be the order of $P$. 
- Assume that $\gcd(m, p)=1$

We want to find $n$ such that $Q=nP$

**Lemma**: $\exists  \ n \ $ such that $ Q = nP \iff mQ = \mathcal{O} $ and $e_m(P,Q) = 1$

## The MOV attack

- https://crypto.stanford.edu/pbc/notes/elliptic/movattack.html
- https://crypto.stackexchange.com/questions/1871/how-does-the-mov-attack-work

Choose embedding degree $k$ such that $E[m] \subset E(\mathbb{F}_{p^k})$

1. Compute the number of points $N = |E(\mathbb{F}_{p^k})|$
2. Choose a random point $T \in E(\mathbb{F}_{p^k}), \ T \not \in E(\mathbb{F}_p)$
3. Compute the order $t$ of $T$
4. let $d = \gcd(t, m)$, let $T' = (\frac t d)T => T'$ has order $d$ which divides $m => T \in E[m]$
    - We can use $N//m$ instead of $d$ but takes more computation
5. Compute 
    - $\alpha = e_m(P, T')$ 
    - $\beta = e_m(Q, T')$
6. Solve $\alpha = \beta^n$ in $\mathbb{F}_{p^k}$ => $n$ mod $d$
7. If there are more $n$'s, crt the results

# Code

In [1]:
p = 1331169830894825846283645180581
a = -35
b = 98
E = EllipticCurve(GF(p), [a,b])
G = E(479691812266187139164535778017, 568535594075310466177352868412)
A = E(1110072782478160369250829345256, 800079550745409318906383650948)

In [2]:
log_p = log(p, 2).n(digits = 10) #a range to search
G_order = G.order()
k = 1
for i in range(1, int(log_p)):
    if pow(p, i, G_order) == 1:
        k = i
        break
               

In [3]:
k

2

In [7]:
def MOV(E, P, Q, k, p):
    """
    Parameters:
    -----------
    
    E: EllipticCurve
        The elliptic curve 
    P: Point
        a point of order l
    Q: Point
        Point such that Q = nP
    k: int
        embedding degree
    p: int
        prime order
        
    Returns: 
    --------
    int: 
        n such that Q = nP
    
    """
    #Step 1 compute the number of points in E(F_{p^k}) => N
    
    
    E_k = E.base_extend(GF(p^k))
    N = E_k.order()
    m = P.order()
    PK = E_k(P)
    QK = E_k(Q)
    
    while True:
        #Step 2 Select  a random point T which is on E_k but not on E
        T = E_k.random_point()        
        try:
            E(*T.xy())
            continue
        except:
            pass
        
        #Step 3 Compute T_prime = (T_order / gcd(T_order, m)) * T;  if T = 0 select another T. Otherwise T_prime has order m
        t = T.order()
        d = gcd(m, t)
        T_prime = (t//d) * T #=>  point of order m
        #print(N//m, t//d)
        if T_prime.is_zero():
            continue
            
        #Step 4 Compute the Weil pairing values. P, Q, and T_prime all have order m
        alpha = PK.weil_pairing(T_prime, m)
        if alpha == 1:
            continue
        beta = QK.weil_pairing(T_prime, m)
        
        #Step 5 Solve dlp for beta = alpha ^ n in F*_{p^k}
        n = beta.log(alpha)
        break
    return n

In [5]:
n = MOV(E, G, A, k, p)

In [6]:
n*G == A

True

In [8]:
def MOV2(E, P, Q, k, p):
    """
    Parameters:
    -----------
    
    E: EllipticCurve
        The elliptic curve 
    P: Point
        a point of order l
    Q: Point
        Point such that Q = nP
    k: int
        embedding degree
    p: int
        prime order
        
    Returns: 
    --------
    int: 
        n such that Q = nP
    
    """
    #Step 1 compute the number of points in E(F_{p^k}) => N
    
    m = P.order()
    E_k = E.base_extend(GF(p**k))
    QK = E_k(Q)
    PK  = E_k(P)
    
    n_list = []
    d_list = []
    least_common_multiple = 1
    
    while least_common_multiple !=m:
        T = E_k.random_point()
        t = T.order()
        d = gcd(t, m)
        if d in d_list:
            continue
        T_prime = t//d * T #T_prime has order d | m => T_prime is in E[m]
        
        alpha = PK.weil_pairing(T_prime, m)
        beta = QK.weil_pairing(T_prime, m) # => both are roots in \mu_d in F_{p^k}
        
        n = beta.log(alpha)  #this gives n mod d
        n_list.append(n)
        d_list.append(d)
        least_common_multiple = lcm(least_common_multiple, d)
    print(n_list)
    return n_list[-1]
    #return n_list[0] if len(n_list) == 1 else crt(n_list, d_list)
    

In [9]:
n_fin = MOV2(E, G, A, k, p)

[29618469991922269]


In [11]:
n_fin * G==A

True

Exiting Sage (CPU time 4m40.27s, Wall time 9m40.99s).

In [ ]:
# Problem curve - Why?

In [253]:
curve = EllipticCurve(GF(22216045306240170001), [15703735179364572855, 10469156786243048570])
gen = curve.gen(1)
n = int(gen.order())
l = randint(1, n - 1)

# Resources